## 31. Python client libraries: `socket`, `requests`, `httpx` / `aiohttp`

*Low‑level*: **`socket`** gives full control but you parse/build HTTP yourself.

*High‑level sync*: **`requests`** abstracts sessions, redirects, cookies.

*High‑level async*: **`httpx.AsyncClient`** or **`aiohttp`** integrate with `asyncio` event loop.

Rule: start with `requests`; drop to async for thousands of concurrent calls.

```python
import requests, asyncio, httpx
print(requests.get('https://httpbin.org/get').status_code)

async def demo():
    async with httpx.AsyncClient() as client:
        r = await client.get('https://httpbin.org/get')
        print(r.status_code)
asyncio.run(demo())
```

### Quick check

1. `requests` is built on top of:
  a. urllib3  b. asyncio

2. True / False `httpx` can operate in both sync and async modes.

<details><summary>Answer key</summary>

1. **a**.
2. **True**.

</details>

## 32. Building a toy HTTP server by hand

To understand HTTP, build a tiny server with raw sockets:
1. Accept connection.
2. Read until `\r\n\r\n` (headers end).
3. Write minimal response.

For production use `http.server` (standard lib) or frameworks.

```python
import socket, threading
def handler(conn):
    data = conn.recv(1024)  # ignore body
    resp = b'HTTP/1.1 200 OK\r\nContent-Length: 5\r\n\r\nHello'
    conn.sendall(resp); conn.close()

srv = socket.socket(); srv.bind(('localhost',8080)); srv.listen()
threading.Thread(target=lambda: handler(srv.accept()[0]), daemon=True).start()
print('curl http://localhost:8080')
```

### Quick check

1. Minimal HTTP response must include:
  a. status line & CRLF  b. JSON body only

2. True / False `http.server` automatically sets `Content-Length` for string responses.

<details><summary>Answer key</summary>

1. **a**.
2. **True**.

</details>

## 33. WSGI vs. ASGI frameworks

* **WSGI** – synchronous gateway (Flask, Django classic). One request → one thread/process.
* **ASGI** – async gateway (FastAPI, Django 3.2+ optional) supporting WebSockets & long‑poll.
Choosing:
* CPU‑light API with many concurrent connections → ASGI.
* Simple CRUD or legacy code → WSGI still fine.

```python
# fastapi_example.py
from fastapi import FastAPI
app = FastAPI()
@app.get('/hello')
async def hello():
    return {'msg':'hi'}
# uvicorn fastapi_example:app --reload
```

### Quick check

1. WebSocket support requires:
  a. ASGI  b. WSGI

2. True / False ASGI apps can run on Gunicorn with `uvicorn.workers.UvicornWorker`.

<details><summary>Answer key</summary>

1. **a**.
2. **True**.

</details>

## 34. JSON serialization pitfalls

Python `json` module:
* **datetime** objects serialize? → need custom `default=str` or ISO format.
* **Decimal** vs. float precision.
* Large ints > 53‑bit lose precision in JavaScript.
* Non‑UTF‑8 bytes invalid – base64 encode binary blobs.

```python
import json, datetime, decimal
data = {'ts': datetime.datetime.utcnow(), 'price': decimal.Decimal('19.99')}
print(json.dumps(data, default=str))
```

### Quick check

1. JavaScript Number max safe integer is 2^?
  a. 53  b. 64

2. True / False `json.dumps` refuses bytes objects by default.

<details><summary>Answer key</summary>

1. **a**.
2. **True**.

</details>

## 35. OpenAPI / Swagger basics

OpenAPI spec (YAML/JSON) documents REST endpoints, schemas, and auth. Tools generate docs (Swagger UI), client SDKs, and server stubs.

Key fields:
`paths`, `components.schemas`, `security`, `servers`.
FastAPI and Flask‑RestX auto‑generate OpenAPI from code annotations.

```yaml
openapi: 3.0.0
info:
  title: Demo
  version: 1.0.0
paths:
  /items/{id}:
    get:
      parameters:
        - in: path
          name: id
          required: true
          schema: {type: integer}
      responses:
        '200': {description: OK}
```

### Quick check

1. Swagger UI renders OpenAPI spec in:
  a. browser  b. Postman only

2. True / False OpenAPI requires code changes to client when spec evolves backward‑compatibly.

<details><summary>Answer key</summary>

1. **a**.
2. **False** – generate new client if needed, but runtime code unchanged.

</details>